In [1]:
import dash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import json
import datetime
import plotly.express as px
import base64

In [2]:
emails = pd.read_csv('email headers.csv', encoding='cp1252')
emails.head()

,From,To,Date,Subject
0,Sven.Flecha@gastech.com.kronos,"Isak.Baza@gastech.com.kronos, Lucas.Alcazar@ga...",1/6/2014 8:39,GT-SeismicProcessorPro Bug Report
1,Kanon.Herrero@gastech.com.kronos,"Felix.Resumir@gastech.com.kronos, Hideki.Cocin...",1/6/2014 8:58,Inspection request for site
2,Bertrand.Ovan@gastech.com.kronos,"Emile.Arpa@gastech.com.kronos, Varro.Awelon@ga...",1/6/2014 9:28,New refueling policies - Effective February 1
3,Valeria.Morlun@gastech.com.kronos,"Dante.Coginian@gastech.com.kronos, Albina.Hafo...",1/6/2014 9:38,Route suggestion for next shift
4,Mat.Bramar@gastech.com.kronos,"Rachel.Pantanal@gastech.com.kronos, Lars.Azada...",1/6/2014 9:49,Upcoming birthdays


In [3]:
# Split up multiple receivers to separate rows
emails = emails.assign(To=emails.To.str.split(', '))
emails = emails.explode('To')
emails

,From,To,Date,Subject
0,Sven.Flecha@gastech.com.kronos,Isak.Baza@gastech.com.kronos,1/6/2014 8:39,GT-SeismicProcessorPro Bug Report
0,Sven.Flecha@gastech.com.kronos,Lucas.Alcazar@gastech.com.kronos,1/6/2014 8:39,GT-SeismicProcessorPro Bug Report
1,Kanon.Herrero@gastech.com.kronos,Felix.Resumir@gastech.com.kronos,1/6/2014 8:58,Inspection request for site
1,Kanon.Herrero@gastech.com.kronos,Hideki.Cocinaro@gastech.com.kronos,1/6/2014 8:58,Inspection request for site
1,Kanon.Herrero@gastech.com.kronos,Inga.Ferro@gastech.com.kronos,1/6/2014 8:58,Inspection request for site
...,...,...,...,...
1169,Albina.Hafon@gastech.com.kronos,Valeria.Morlun@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today
1169,Albina.Hafon@gastech.com.kronos,Cecilia.Morluniau@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today
1169,Albina.Hafon@gastech.com.kronos,Henk.Mies@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today
1169,Albina.Hafon@gastech.com.kronos,Dylan.Scozzese@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today


In [4]:
# Separate the date and time in different columns
emails['Time'] = pd.to_datetime(emails['Date']).dt.time
emails['Date'] = pd.to_datetime(emails['Date']).dt.date
emails

,From,To,Date,Subject,Time
0,Sven.Flecha@gastech.com.kronos,Isak.Baza@gastech.com.kronos,2014-01-06,GT-SeismicProcessorPro Bug Report,08:39:00
0,Sven.Flecha@gastech.com.kronos,Lucas.Alcazar@gastech.com.kronos,2014-01-06,GT-SeismicProcessorPro Bug Report,08:39:00
1,Kanon.Herrero@gastech.com.kronos,Felix.Resumir@gastech.com.kronos,2014-01-06,Inspection request for site,08:58:00
1,Kanon.Herrero@gastech.com.kronos,Hideki.Cocinaro@gastech.com.kronos,2014-01-06,Inspection request for site,08:58:00
1,Kanon.Herrero@gastech.com.kronos,Inga.Ferro@gastech.com.kronos,2014-01-06,Inspection request for site,08:58:00
...,...,...,...,...,...
1169,Albina.Hafon@gastech.com.kronos,Valeria.Morlun@gastech.com.kronos,2014-01-17,RE: Traffic advisory for today,20:46:00
1169,Albina.Hafon@gastech.com.kronos,Cecilia.Morluniau@gastech.com.kronos,2014-01-17,RE: Traffic advisory for today,20:46:00
1169,Albina.Hafon@gastech.com.kronos,Henk.Mies@gastech.com.kronos,2014-01-17,RE: Traffic advisory for today,20:46:00
1169,Albina.Hafon@gastech.com.kronos,Dylan.Scozzese@gastech.com.kronos,2014-01-17,RE: Traffic advisory for today,20:46:00


In [5]:
# Read the employee records, check for employment types
records = pd.read_csv('EmployeeRecords.csv')

# Some processing for displaying the data later on
records = records.fillna('None')
double_email = records[records['LastName'] == 'Sanjorge Jr.']
records = records.append(double_email, ignore_index=True)
records.at[54, 'EmailAddress'] = 'Sten.Sanjorge Jr.@gastech.com.tethys'

records.tail()

,LastName,FirstName,BirthDate,BirthCountry,Gender,CitizenshipCountry,CitizenshipBasis,CitizenshipStartDate,PassportCountry,PassportIssueDate,PassportExpirationDate,CurrentEmploymentType,CurrentEmploymentTitle,CurrentEmploymentStartDate,EmailAddress,MilitaryServiceBranch,MilitaryDischargeType,MilitaryDischargeDate
50,Resumir,Felix,30-06-59,Tethys,Male,Tethys,BirthNation,30-06-59,Tethys,20-03-05,19-03-15,Security,Security Group Manager,15-09-03,Felix.Resumir@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,01-10-97
51,Bodrogi,Loreto,17-04-89,Kronos,Male,Kronos,BirthNation,17-04-89,None,None,None,Security,Site Control,17-08-13,Loreto.Bodrogi@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,01-10-08
52,Cocinaro,Hideki,25-12-80,Tethys,Male,Tethys,BirthNation,25-12-80,Tethys,25-05-13,24-05-23,Security,Site Control,01-01-10,Hideki.Cocinaro@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,01-10-09
53,Ferro,Inga,17-06-89,Kronos,Female,Kronos,BirthNation,17-06-89,None,None,None,Security,Site Control,11-01-13,Inga.Ferro@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,01-10-12
54,Sanjorge Jr.,Sten,29-08-48,Tethys,Male,Tethys,BirthNation,29-08-48,Tethys,21-04-13,20-04-23,Executive,President/CEO,09-11-90,Sten.Sanjorge Jr.@gastech.com.tethys,None,None,None


In [7]:
def create_json(emails, records, day: int, departments=['Administration', 'Engineering', 'Executive', 'Facilities', 'Information Technology', 'Security']):
    """Given a split-out dataframe with emails (1 from, 1 to, and a date) and , a json is created that can be
    used as input for the Cytoscape network graph."""

    # Extract the from and to emails
    emails_from = emails.From.tolist()
    emails_to = emails.To.tolist()

    # Create a list with all appearing emails and remove duplicates
    nodes = emails_from + emails_to
    nodes = list(dict.fromkeys(nodes))

    nodes_df = pd.DataFrame(nodes, columns=['EmailAddress'])
    records_mod = records[['EmailAddress', 'CurrentEmploymentType']]
    # There is one line missing in the records compared to emails: Executive Sten Sanjorge Jr. has two email addresses, one ending in .tethys (not in the records) and one ending in .kronos, suspicious?
    records_mod = records_mod.append({'EmailAddress': 'Sten.Sanjorge Jr.@gastech.com.tethys', 'CurrentEmploymentType': 'Executive'}, ignore_index=True)

    merged = nodes_df.merge(records_mod, how='inner', on=['EmailAddress'])
    merged = merged.sort_values(by=['CurrentEmploymentType'])
    merged = merged[merged['CurrentEmploymentType'].isin(departments)]
    merged_list = merged.values.tolist()

    # Create the formatted text for the nodes
    nodes_list = []
    for node in merged_list:
        name = node[0].split('@')[0].split('.')
        text = "{\"data\": {\"id\": \"" + node[0] + "\", \"label\": \"" + name[0] + " " + name[1] + "\"}, \"classes\": \"" + node[1] + "\"}"
        nodes_list.append(text)

    # Create a list of from - to email pairs
    pairs = []
    for i in range(len(emails_from)):
        if (emails_from[i] in merged['EmailAddress'].values.tolist()) and (emails_to[i] in merged['EmailAddress'].values.tolist()):
            pairs.append([emails_from[i], emails_to[i]])
        else:
            pass
    # Compute weights and add them to the pairs
    weights = []
    for pair in pairs:
        weights.append(pairs.count(pair))
    count = 0
    for pair in pairs:
        pair.append(weights[count])
        count += 1
    # Remove duplicates
    unique = []
    for pair in pairs:
        if pair not in unique:
            unique.append(pair)
        else:
            pass
    # Create the correct format for the links
    links_list = []
    for link in unique:
        text = "{\"data\": {\"source\": \"" + link[0] + "\", \"target\": \"" + link[1] + "\", \"weight\": " + str(link[2]) + "}}"
        links_list.append(text)

    # Write to the json file
    file_name = './network_data_day' + str(day) + '.json'

    with open(file_name, 'w') as network_data:
        z = 0
        network_data.writelines("[")
        while z < (len(nodes_list)):
            network_data.writelines(nodes_list[z] + ",")
            z += 1
        z = 0
        while z < (len(links_list) - 1):
            network_data.writelines(links_list[z] + ",")
            z += 1
        network_data.writelines(links_list[-1])
        network_data.writelines("]")

In [8]:
# Filter out emails from the days
emails_day1 = emails[emails['Date'] == datetime.date(2014, 1, 6)]
emails_day2 = emails[emails['Date'] == datetime.date(2014, 1, 7)]
emails_day3 = emails[emails['Date'] == datetime.date(2014, 1, 8)]
emails_day4 = emails[emails['Date'] == datetime.date(2014, 1, 9)]
emails_day5 = emails[emails['Date'] == datetime.date(2014, 1, 10)]
emails_day6 = emails[emails['Date'] == datetime.date(2014, 1, 13)]
emails_day7 = emails[emails['Date'] == datetime.date(2014, 1, 14)]
emails_day8 = emails[emails['Date'] == datetime.date(2014, 1, 15)]
emails_day9 = emails[emails['Date'] == datetime.date(2014, 1, 16)]
emails_day10 = emails[emails['Date'] == datetime.date(2014, 1, 17)]

In [9]:
create_json(emails_day1, records, 1)
create_json(emails_day2, records, 2)
create_json(emails_day3, records, 3)
create_json(emails_day4, records, 4)
create_json(emails_day5, records, 5)
create_json(emails_day6, records, 6)
create_json(emails_day7, records, 7)
create_json(emails_day8, records, 8)
create_json(emails_day9, records, 9)
create_json(emails_day10, records, 10)

In [10]:
app = dash.Dash(__name__)

with open('network_data_day1.json', 'r') as f:
    data_day1 = json.load(f)
with open('network_data_day2.json', 'r') as f:
    data_day2 = json.load(f)
with open('network_data_day3.json', 'r') as f:
    data_day3 = json.load(f)
with open('network_data_day4.json', 'r') as f:
    data_day4 = json.load(f)
with open('network_data_day5.json', 'r') as f:
    data_day5 = json.load(f)
with open('network_data_day6.json', 'r') as f:
    data_day6 = json.load(f)
with open('network_data_day7.json', 'r') as f:
    data_day7 = json.load(f)
with open('network_data_day8.json', 'r') as f:
    data_day8 = json.load(f)
with open('network_data_day9.json', 'r') as f:
    data_day9 = json.load(f)
with open('network_data_day10.json', 'r') as f:
    data_day10 = json.load(f)

# Set the colorscale to extract colors for the network graph from
colors = px.colors.qualitative.Set3
# Import the legend image and encode it
image_filename = 'network_legend.PNG'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

In [11]:
stylesheet = [
    {
        'selector': 'node',
        'style': {
            'label': 'data(label)'
        }
    },
    {
        'selector': 'edge', # Add a toggle to turn on edge directions
        'style': {
            'mid-target-arrow-color': '#C5D3E2',
            'mid-target-arrow-shape': 'triangle',
            'arrow-scale': 2
        }
    },
    {
        'selector': '[weight > 3]',
        'style': {
            'line-color': 'red'
        }
    },
    {
        'selector': '.Information',
        'style': {
            'background-color': colors[0]
        }
    }
    ,
    {
        'selector': '.Executive',
        'style': {
            'background-color': colors[1]
        }
    },
    {
        'selector': '.Engineering',
        'style': {
            'background-color': colors[2]
        }
    },
    {
        'selector': '.Security',
        'style': {
            'background-color': colors[3]
        }
    },
    {
        'selector': '.Facilities',
        'style': {
            'background-color': colors[4]
        }
    },
    {
        'selector': '.Administration',
        'style': {
            'background-color': colors[5]
        }
    }
]

In [12]:
app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label(['Select a part of the day to filter'],
                       style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px'}),
            dcc.RadioItems(
                id='my-radio',
                options=[{'label': 'Morning', 'value': 'morning'},
                         {'label': 'Afternoon', 'value': 'afternoon'},
                         {'label': 'Evening', 'value': 'evening'}]
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.Label(['Select department(s) to display'],
                       style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px'}),
            dcc.Checklist(
                id='my-checklist',
                options=[{'label': 'Administration', 'value': 'Administration'},
                         {'label': 'Engineering', 'value': 'Engineering'},
                         {'label': 'Executive', 'value': 'Executive'},
                         {'label': 'Facilities', 'value': 'Facilities'},
                         {'label': 'Information Technology', 'value': 'Information Technology'},
                         {'label': 'Security', 'value': 'Security'}],
                value=['Administration', 'Engineering', 'Executive', 'Facilities', 'Information Technology', 'Security'],
                labelStyle={'display': 'inline-block'}
            ),
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={'width': '87%'})
        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),
    html.Div([
        html.Label(['2014-01-06'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-07'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-08'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-09'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-10'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'})
    ]),
    html.Div([
        cyto.Cytoscape(
            id='cytoscape-v1',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day1,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v2',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day2,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v3',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day3,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v4',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day4,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v5',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day5,
            stylesheet=stylesheet
        )
    ]),
    html.Div([
        html.P(id='tapNodeData-cyto-v1',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v2',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v3',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v4',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v5',
               style={'width': '20%', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Label(['2014-01-13'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-14'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-15'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-16'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
        html.Label(['2014-01-17'],
                   style={'font-weight': 'bold', 'text-align': 'center', 'font-size': '22px',
                          'width': '20%', 'display': 'inline-block'}),
    ]),
    html.Div([
        cyto.Cytoscape(
            id='cytoscape-v6',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day6,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v7',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day7,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v8',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day8,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v9',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day9,
            stylesheet=stylesheet
        ),
        cyto.Cytoscape(
            id='cytoscape-v10',
            layout={'name': 'circle'},
            style={'width': '20%', 'height': '300px', 'display': 'inline-block'},
            elements=data_day10,
            stylesheet=stylesheet
        )
    ]),
    html.Div([
        html.P(id='tapNodeData-cyto-v6',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v7',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v8',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v9',
               style={'width': '20%', 'display': 'inline-block'}),
        html.P(id='tapNodeData-cyto-v10',
               style={'width': '20%', 'display': 'inline-block'})
    ])
])

In [13]:
@app.callback(
    [dash.dependencies.Output(component_id='cytoscape-v1', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v2', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v3', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v4', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v5', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v6', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v7', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v8', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v9', component_property='elements'),
     dash.dependencies.Output(component_id='cytoscape-v10', component_property='elements')],
    [dash.dependencies.Input(component_id='my-radio', component_property='value'),
     dash.dependencies.Input(component_id='my-checklist', component_property='value')])

def update_output(radio_value, checklist_values):

    emails_day1 = emails[emails['Date'] == datetime.date(2014, 1, 6)]
    emails_day2 = emails[emails['Date'] == datetime.date(2014, 1, 7)]
    emails_day3 = emails[emails['Date'] == datetime.date(2014, 1, 8)]
    emails_day4 = emails[emails['Date'] == datetime.date(2014, 1, 9)]
    emails_day5 = emails[emails['Date'] == datetime.date(2014, 1, 10)]
    emails_day6 = emails[emails['Date'] == datetime.date(2014, 1, 13)]
    emails_day7 = emails[emails['Date'] == datetime.date(2014, 1, 14)]
    emails_day8 = emails[emails['Date'] == datetime.date(2014, 1, 15)]
    emails_day9 = emails[emails['Date'] == datetime.date(2014, 1, 16)]
    emails_day10 = emails[emails['Date'] == datetime.date(2014, 1, 17)]

    if radio_value == 'morning':
        emails_day1 = emails_day1[emails_day1['Time'] <= datetime.time(12, 0, 0)]
        emails_day2 = emails_day2[emails_day2['Time'] <= datetime.time(12, 0, 0)]
        emails_day3 = emails_day3[emails_day3['Time'] <= datetime.time(12, 0, 0)]
        emails_day4 = emails_day4[emails_day4['Time'] <= datetime.time(12, 0, 0)]
        emails_day5 = emails_day5[emails_day5['Time'] <= datetime.time(12, 0, 0)]
        emails_day6 = emails_day6[emails_day6['Time'] <= datetime.time(12, 0, 0)]
        emails_day7 = emails_day7[emails_day7['Time'] <= datetime.time(12, 0, 0)]
        emails_day8 = emails_day8[emails_day8['Time'] <= datetime.time(12, 0, 0)]
        emails_day9 = emails_day9[emails_day9['Time'] <= datetime.time(12, 0, 0)]
        emails_day10 = emails_day10[emails_day10['Time'] <= datetime.time(12, 0, 0)]

    if radio_value == 'afternoon':
        emails_day1 = emails_day1[(emails_day1['Time'] > datetime.time(12, 0, 0)) & (emails_day1['Time'] <= datetime.time(18, 0, 0))]
        emails_day2 = emails_day2[(emails_day2['Time'] > datetime.time(12, 0, 0)) & (emails_day2['Time'] <= datetime.time(18, 0, 0))]
        emails_day3 = emails_day3[(emails_day3['Time'] > datetime.time(12, 0, 0)) & (emails_day3['Time'] <= datetime.time(18, 0, 0))]
        emails_day4 = emails_day4[(emails_day4['Time'] > datetime.time(12, 0, 0)) & (emails_day4['Time'] <= datetime.time(18, 0, 0))]
        emails_day5 = emails_day5[(emails_day5['Time'] > datetime.time(12, 0, 0)) & (emails_day5['Time'] <= datetime.time(18, 0, 0))]
        emails_day6 = emails_day6[(emails_day6['Time'] > datetime.time(12, 0, 0)) & (emails_day6['Time'] <= datetime.time(18, 0, 0))]
        emails_day7 = emails_day7[(emails_day7['Time'] > datetime.time(12, 0, 0)) & (emails_day7['Time'] <= datetime.time(18, 0, 0))]
        emails_day8 = emails_day8[(emails_day8['Time'] > datetime.time(12, 0, 0)) & (emails_day8['Time'] <= datetime.time(18, 0, 0))]
        emails_day9 = emails_day9[(emails_day9['Time'] > datetime.time(12, 0, 0)) & (emails_day9['Time'] <= datetime.time(18, 0, 0))]
        emails_day10 = emails_day10[(emails_day10['Time'] > datetime.time(12, 0, 0)) & (emails_day10['Time'] <= datetime.time(18, 0, 0))]

    if radio_value == 'evening':
        emails_day1 = emails_day1[emails_day1['Time'] > datetime.time(18, 0, 0)]
        emails_day2 = emails_day2[emails_day2['Time'] > datetime.time(18, 0, 0)]
        emails_day3 = emails_day3[emails_day3['Time'] > datetime.time(18, 0, 0)]
        emails_day4 = emails_day4[emails_day4['Time'] > datetime.time(18, 0, 0)]
        emails_day5 = emails_day5[emails_day5['Time'] > datetime.time(18, 0, 0)]
        emails_day6 = emails_day6[emails_day6['Time'] > datetime.time(18, 0, 0)]
        emails_day7 = emails_day7[emails_day7['Time'] > datetime.time(18, 0, 0)]
        emails_day8 = emails_day8[emails_day8['Time'] > datetime.time(18, 0, 0)]
        emails_day9 = emails_day9[emails_day9['Time'] > datetime.time(18, 0, 0)]
        emails_day10 = emails_day10[emails_day10['Time'] > datetime.time(18, 0, 0)]

    try:
        create_json(emails_day1, records, 1, checklist_values)
        with open('network_data_day1.json', 'r') as f:
            elements_day1 = json.load(f)
    except:
        elements_day1 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day2, records, 2, checklist_values)
        with open('network_data_day2.json', 'r') as f:
            elements_day2 = json.load(f)
    except:
        elements_day2 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day3, records, 3, checklist_values)
        with open('network_data_day3.json', 'r') as f:
            elements_day3 = json.load(f)
    except:
        elements_day3 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day4, records, 4, checklist_values)
        with open('network_data_day4.json', 'r') as f:
            elements_day4 = json.load(f)
    except:
        elements_day4 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day5, records, 5, checklist_values)
        with open('network_data_day5.json', 'r') as f:
            elements_day5 = json.load(f)
    except:
        elements_day5 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day6, records, 6, checklist_values)
        with open('network_data_day6.json', 'r') as f:
            elements_day6 = json.load(f)
    except:
        elements_day6 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day7, records, 7, checklist_values)
        with open('network_data_day7.json', 'r') as f:
            elements_day7 = json.load(f)
    except:
        elements_day7 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day8, records, 8, checklist_values)
        with open('network_data_day8.json', 'r') as f:
            elements_day8 = json.load(f)
    except:
        elements_day8 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day9, records, 9, checklist_values)
        with open('network_data_day9.json', 'r') as f:
            elements_day9 = json.load(f)
    except:
        elements_day9 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]
    try:
        create_json(emails_day10, records, 10, checklist_values)
        with open('network_data_day10.json', 'r') as f:
            elements_day10 = json.load(f)
    except:
        elements_day10 = [{'data': {'id': 'one', 'label': 'Nothing to display'}}]

    return elements_day1, elements_day2, elements_day3, elements_day4, elements_day5, elements_day6, elements_day7, elements_day8, elements_day9, elements_day10

In [14]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v1', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v1', component_property='tapNodeData')]
)

def displayTap_v1(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Mail:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [15]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v2', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v2', component_property='tapNodeData')]
)

def displayTap_v2(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [16]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v3', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v3', component_property='tapNodeData')]
)

def displayTap_v3(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [17]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v4', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v4', component_property='tapNodeData')]
)

def displayTap_v4(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [18]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v5', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v5', component_property='tapNodeData')]
)

def displayTap_v5(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [19]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v6', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v6', component_property='tapNodeData')]
)

def displayTap_v6(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [20]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v7', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v7', component_property='tapNodeData')]
)

def displayTap_v7(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [21]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v8', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v8', component_property='tapNodeData')]
)

def displayTap_v8(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [22]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v9', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v9', component_property='tapNodeData')]
)

def displayTap_v9(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [23]:
@app.callback(
    dash.dependencies.Output(component_id='tapNodeData-cyto-v10', component_property='children'),
    [dash.dependencies.Input(component_id='cytoscape-v10', component_property='tapNodeData')]
)

def displayTap_v10(data):
    email = data['id']
    record = records[records['EmailAddress'] == email].reset_index()

    return "Name: " + data['label'] + ', Email:' + email + ', Country: ' + record['CitizenshipCountry'][0] + ', Military: ' + record['MilitaryServiceBranch'][0]

In [25]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Next steps:
- Add other days as well.
- Add checkboxes to select departments.
- Add radioitems (or maybe checkboxes?) to select on graph bigger.
- On click, remove a node and it's edges.